# 在class里定义Property
## 使用@property

In [25]:
class Cls(object):
    data = 'class data'
    @property
    def prop(self):
        return 'class property value'
    
obj = Cls()
print('obj has no instance attribute in its __dict__ now:', vars(obj))
obj.data = 'object data'
print('obj has instance attribute in its __dict__ now:', vars(obj))
print('the data attribute of Cls is untouched:', Cls.data)
obj.__dict__['prop'] = 'object attribute'
print('the class property can override instance attribute:', obj.prop)
Cls.prop = 'not property any more'
print('instance attribute comes first when no property exits:', obj.prop)


obj has no instance attribute in its __dict__ now: {}
obj has instance attribute in its __dict__ now: {'data': 'object data'}
the data attribute of Cls is untouched: class data
the class property can override instance attribute: class property value
instance attribute comes first when no property exits: object attribute


## 使用`class property(fget=None, fset=None, fdel=None, doc=None)`
property在上面被当作修饰器使用, 可实质上它是一个class. 

In [31]:
class Cls(object):
    __prop = 'class property'
    def get_prop(self):
        return self.__prop
    def set_prop(self,value):
        self.__prop = value
        
    prop = property(get_prop, set_prop)
    
obj = Cls()
print(obj.prop)
print(vars(obj))
obj.prop = 'instance attribute'
print(vars(obj))
print(obj.prop)
print(Cls.prop)

class property
{}
{'_Cls__prop': 'instance attribute'}
instance attribute


In [36]:
class Cls(object):
    pass
obj = Cls()
print(obj.__dict__)
obj.age = 20
print(obj.__dict__)

{}
{'age': 20}


In [46]:
class Cls(object):
    __slots__ = ('name')
    def hello(self):
        print('hello.')
obj = Cls()
#print(vars(obj))
obj.name = 20
#print(obj.__dict__)
print(vars(type(obj)))
obj.__get = Cls

{'name': <member 'name' of 'Cls' objects>, '__doc__': None, '__slots__': 'name', 'hello': <function Cls.hello at 0x7fd135ed3d08>, '__module__': '__main__'}


AttributeError: 'Cls' object has no attribute '__get'

In [53]:
class Cls(object):
    __slots__=('age')
    name = 'class name'
    def __init__(self):
        self.age = 20
    def hello(self):
        print('hello.')
obj = Cls()
print(obj.name)
obj.name = 'name'

class name


AttributeError: 'Cls' object attribute 'name' is read-only

In [63]:
class Foo():
    name = 'Foo class attribute'
print('class attribute:', Foo.__dict__)
print()
foo = Foo()
foo.name = 'foo instance attribute'
print('instance attribute:', foo.__dict__)

class attribute: {'fn': <function Foo.fn at 0x7fd135ec8ea0>, '__weakref__': <attribute '__weakref__' of 'Foo' objects>, '__dict__': <attribute '__dict__' of 'Foo' objects>, '__doc__': None, '__module__': '__main__', 'name': 'Foo class attribute'}

instance attribute: {'name': 'foo instance attribute'}


In [66]:
class Foo():
    def __init__(self, name):
        self.name = name
    
foo = Foo('I do not want to be changed')
print('foo.name = ', foo.name)
foo.name = 'Unluckily, I can be changed'
print('foo.name = ', foo.name)

foo.name =  I do not want to be changed
foo.name =  Unluckily, I can be changed


In [69]:
class Foo():
    def __init__(self, name):
        self.__name = name
    
    @property
    def name(self):
        return self.__name
    
foo = Foo('I do not want to be changed')
print('foo.name = ', foo.name)
foo.name = 'Luckily, I really can not be changed'

foo.name =  I do not want to be changed


AttributeError: can't set attribute

In [73]:
class Item():
    def __init__(self, category, count, price):
        self.cat = category
        self.count = count
        self.price = price
        
    def get_bill(self):
        return self.count * self.price
    
item = Item('Bread', 1, 10)

In [74]:
invalid_item1 = Item('Bread', -1, 10)
invalid_item2 = Item('Bread', 1, -10)

In [91]:
class Item():
    def __init__(self, category, count, price):
        self.__cat = category # attribute
        self.count = count # property
        self.price = price # property
    @property    
    def bill(self):
        return self.__count * self.__price

    @property
    def cat(self):
        return self.__cat
    
    @property
    def count(self):
        return self.__dict__['count']
    @count.setter
    def count(self, value):
        if value < 0:
            raise ValueError('count can not be minus: %r'%(value))
        self.__dict__['count'] = value
        
    @property
    def price(self):
        return self.__dict__['price']
    
    @price.setter
    def price(self, value):
        if value < 0:
            raise ValueError('price can not be minus: %r'%(value))
        self.__dict__['price'] = value


In [92]:
item = Item('Bread', 1, 10)
item.price = 20
item.count = 2
print(item.price)

20


In [83]:
item = Item('Bread', 1, -10)

ValueError: price can not be minus: -10

In [93]:
item.price = -10

ValueError: price can not be minus: -10

In [94]:
class Item():
    def __init__(self, category, count, price):
        self.__cat = category # attribute
        self.count = count # property
        self.price = price # property

    def get_bill(self):
        return self.__count * self.__price

    def get_cat(self):
        return self.__cat
    
    def get_count(self):
        return self.__dict__['count']

    def set_count(self, value):
        if value < 0:
            raise ValueError('count can not be minus: %r'%(value))
        self.__dict__['count'] = value
        
    def get_price(self):
        return self.__dict__['price']
    
    def set_price(self, value):
        if value < 0:
            raise ValueError('price can not be minus: %r'%(value))
        self.__dict__['price'] = value
    bill = property(get_bill)
    cat = property(get_cat)
    count = property(get_count, set_count)
    price = property(get_price, set_price)
    

In [95]:
item = Item('Bread', 1, 10)
item.price = 20
item.count = 2
print(item.price)

20


In [96]:
item = Item('Bread', 1, -10)

ValueError: price can not be minus: -10

In [101]:
def readonly_prop(storage_name):
    def getter(instance):
        return instance.__dict__[storage_name]
    return property(getter)
def positive_mutable_prop(storage_name):
    def getter(instance):
        return instance.__dict__[storage_name]
    def setter(instance, value):
        if value < 0:
            raise ValueError('%s can not be minus: %r'%(storage_name, value))
        instance.__dict__[storage_name] = value
    return property(getter, setter)

class Item():
    def __init__(self, category, count, price):
        self.__cat = category # attribute
        self.count = count # property
        self.price = price # property
   
    cat = readonly_prop('__cat')
    count = positive_mutable_prop('count')
    price = positive_mutable_prop('price')
    

In [98]:
item = Item('Bread', 1, 10)
item.price = 20
item.count = 2
print(item.price)

20


In [102]:
item = Item('Bread', 1, -10)

ValueError: price can not be minus: -10

In [104]:
class Foo():
    name = 'Foo'
    
foo = Foo()
foo.name = 'foo'
codes = ['Foo.name', 'foo.name']
for code in codes:
    print(code, '=', eval(code))

Foo.name = Foo
foo.name = foo


In [106]:
class Foo():
    @property
    def name(self):
        return 'Foo'
    
foo = Foo()
foo.__dict__['name'] = 'foo'
codes = ['Foo.name', 'foo.name']
for code in codes:
    print(code, '=', eval(code))

Foo.name = <property object at 0x7fd135e7ecc8>
foo.name = Foo


In [109]:
class Foo():
    # 无修饰
    def instance_method(self): #传入的第一个参数是self, 即instance本身
        print('the first argument of instance_method:', self)
        
    @classmethod
    def class_method(cls): # 传入的第一个参数是class
        print('the first argument of class_method:', cls)
        
    @staticmethod
    def static_method(): # 没有默认的首位参数, 只有自定义参数
        print('the first argument of static_method:')
        
foo = Foo()
foo.instance_method()
Foo.class_method()
foo.class_method()
Foo.static_method()
foo.static_method()
try:
    Foo.instance_method()
except:
    print('instance method can not be accessed through class.')

the first argument of instance_method: <__main__.Foo object at 0x7fd135ec3b38>
the first argument of class_method: <class '__main__.Foo'>
the first argument of class_method: <class '__main__.Foo'>
the first argument of static_method:
the first argument of static_method:
instance method can not be accessed through class


In [140]:
class cls_decorator(object):
    def __init__(self):
        print('__init__ is called')
        
    def __new__(cls, fn = None):
        if fn is not None and callable(fn):
            def new_fn(*args, **kwargs):
                print('decorated', fn.__name__, 'is called.')
                result = fn(*args, **kwargs)
                return result
            return new_fn
        else:
            return super().__new__(cls)
@cls_decorator
def fn(a, b, c):
    return 3

print('As a decorator:', fn(1, 2, c = 3))
print('As a class:', cls_decorator())

decorated fn is called.
As a decorator: 3
__init__ is called
As a class: <__main__.cls_decorator object at 0x7fd135e774a8>


In [126]:
class Foo():
    @property
    def name(self):
        return 'Foo'
    def get_age(self):
        return 10
    age = property(get_age)

print(Foo.name, Foo.age)
foo = Foo()
print(foo.name, foo.age)

<property object at 0x7fd135e604f8> <property object at 0x7fd135ef51d8>
Foo 10
